L'objectif de ce notebook est d'optimiser les prix pour une famille de produits. Il prend en entré un data frame et publie des prix pour chaque produit dans le data frame. 
Il est important de mettre seulement une famille de produit et une periode de temps (E ou F) par simulation. En revanche, il est possible d'inclure des ventes sur plusieurs années. Il est egalement recommendé (mais pas necessaire) de faire une simulation par PO (essentiels, essenties star, historique)
Le notebook se base sur l'article suivant : Aviv, Yossi, and Amit Pazgal. "Optimal pricing of seasonal products in the presence of forward-looking consumers." Manufacturing & service operations management 10.3 (2008): 339-359.<http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.476.8707&rep=rep1&type=pdf>

Description algo:
L'algo considère que l'attractivité du produit se dégrade dans le temps suivant une loi exp inv. 
On modélise cette degradation. On simule ensuite une population de consommateur (loi normale) centré sur l'appetance initial (aux alentours du prix initial). quand on augmente le prix, moins de consommateur dans la 
population seront appetant mais la marge augment (inv. baisse). On calul les qty et les marges pour chaque semaine (considérant que l'appetance baisse d'une semaine à l'autre). On prend le prix qui maximise la marge.



Optim Prix
    prix_init:=Calcul prix initial #prix semaine lancement
    alpha_:=Calcul courbe de vente# degradation vente par semaine (en l'occurence degradation appetances/sem)
    appetances:=Calcul appetance (combien des consomateurs voudraient payer pour ce produit)#Normalement un paramètre
    prix: optimisation prix (array prix possibles, prix_init, alpha_,population)

#NB : appetances est la valeur moyenne du produit en euro pour une loi normale de consommateurs

In [ ]:
# Exemple de requête pour récuperer le data frame d'entrée

'''select tic.CODE_REFERENCE,PRIX_ACTUEL_AVEC_OP,tic.DATE_VENTE, count(*) as sum_ ,min(tic.DATE_VENTE) as mindate,col.ANNEE_COLLECTION,SAISON_INI,PERIODE_INI, lib_val_po2,DATE_DEBUT_COLLECTION,DATE_FIN_COLLECTION ,

avg(
KPI_MNT_VENTES_HT_DEV
-KPI_MNT_RETOURS_HT_DEV
-KPI_PRMP_HT_EUR
-nvl(KPI_PORTE_DISPEO_HT_DEV, 0)
-KPI_REDEVANCE_BEELINE_HT_DEV
+KPI_MNT_RETOUCHES_HT_DEV
  
  ) as marge
from 

DATAWH.FD_VENTES_TICKET_LIGNE tic


inner join DATAWH.DD_RCT_PRODUITS rc on rc.CODE_REFERENCE=tic.CODE_REFERENCE
inner join DATAWH.DD_magasins ddm on  ddm.CODE_MAGASIN =  tic.CODE_MAGASIN 
inner join DATAWH.DD_RCT_PRODUITS_CRITERES pc on pc.CODE_REFERENCE = rc.CODE_REFERENCE
inner join DATAWH.DD_COLLECTION col on col.CODE_COLLECTION = PERIODE_INI and col.ANNEE_COLLECTION =rc.ANNEE_COLLECTION
where 
CODE_ACTION_MARKETING = 0 and tic.CODE_DEVISE= 'EUR' and tic.CODE_REFERENCE!='ND'and SAISON_INI='E'// and KPI_MNT_PRIX_ACTU_TTC_DEV>0 and annee_collection is not NULL and saison is not NULL
and tic.DATE_VENTE>current_date - 1095 and PRIX_ACTUEL_HORS_OP>0  and PRIX_ACTUEL_HORS_OP >0 and (PRIX_ACTUEL_HORS_OP-PRIX_ACTUEL_AVEC_OP)/PRIX_ACTUEL_HORS_OP<=0.7// and BOOL_EN_PROMO='Oui' 
and (ddm.CODE_TYPE_SITE IN ('M','7')) and CODE_FAMILLE= 'F610' and code_pays='FR'  and (col.ANNEE_COLLECTION=2020 or col.ANNEE_COLLECTION=2019 or col.ANNEE_COLLECTION=2018 or col.ANNEE_COLLECTION=2017)//and (PRIX_ACTUEL_HORS_OP-PRIX_ACTUEL_AVEC_OP)/PRIX_ACTUEL_HORS_OP =0
group by tic.CODE_REFERENCE,tic.DATE_VENTE,col.ANNEE_COLLECTION,SAISON_INI,PERIODE_INI,PRIX_ACTUEL_AVEC_OP,lib_val_po2,DATE_DEBUT_COLLECTION,DATE_FIN_COLLECTION  //or ddm.CODE_TYPE_SITE = '7'
'''


# Imports

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
#import statsmodels.formula.api as smf

# Lecture des données

In [1331]:
df = pd.read_csv('result_veste.tsv',encoding='ISO-8859-1',delimiter='\t',parse_dates=['DATE_VENTE'])#,,
#df=df[df.DATE_VENTE>="01-01-2020"]# DEBUT DES VENTES

df=df[df.MARGE>0]
df_agg=df.groupby("CODE_REFERENCE").agg({"PRIX_ACTUEL_AVEC_OP":"mean","ANNEE_COLLECTION":"first","SAISON_INI":"first","PERIODE_INI":"first","LIB_VAL_PO2":"first","MARGE":"mean","DATE_DEBUT_COLLECTION":"first","DATE_FIN_COLLECTION":"first","SUM_":"sum"})
df_agg=df_agg.reset_index(drop=False)
df.head()

,CODE_REFERENCE,PRIX_ACTUEL_AVEC_OP,DATE_VENTE,SUM_,MINDATE,ANNEE_COLLECTION,SAISON_INI,PERIODE_INI,LIB_VAL_PO2,DATE_DEBUT_COLLECTION,DATE_FIN_COLLECTION,MARGE
784,507844,17.99,2020-01-15,486,2020-01-15 00:00:00.000,2017,E,E3,Essentiel Star,2017-04-24 00:00:00.000,2017-06-04 00:00:00.000,3.91
795,507844,17.99,2020-01-18,486,2020-01-18 00:00:00.000,2017,E,E3,Essentiel Star,2017-04-24 00:00:00.000,2017-06-04 00:00:00.000,3.79
927,507844,14.39,2020-01-25,162,2020-01-25 00:00:00.000,2017,E,E2,Essentiel,2017-03-06 00:00:00.000,2017-04-23 00:00:00.000,0.79
1401,517523,17.00,2020-05-03,36,2020-05-03 00:00:00.000,2018,E,E3,Essentiel,2018-04-23 00:00:00.000,2018-06-03 00:00:00.000,1.08
1403,517523,17.00,2020-05-12,36,2020-05-12 00:00:00.000,2018,E,E3,Essentiel,2018-04-23 00:00:00.000,2018-06-03 00:00:00.000,1.08


# Formattage des dates (time unit=week)

In [1332]:
df["DATE_DEBUT_COLLECTION"]=pd.to_datetime(df["DATE_DEBUT_COLLECTION"])
df["week"] = np.round((((df["DATE_VENTE"]-pd.to_datetime("01-01-2020"))/7).dt.days)/7).astype(int)

df=df[df.week>=0]
df["week"].describe() 

count    3738.000000
mean        1.165062
std         0.933880
min         0.000000
25%         0.000000
50%         1.000000
75%         2.000000
max         3.000000
Name: week, dtype: float64

# Calcul du prix fab+port

In [1333]:
unique_rc=np.unique(df["CODE_REFERENCE"])
#la colonne SUM fait reference aux qty de ventes
#rc_SUM_max : pour chaque rc, le maximum des ventes durant les k prochaines semaines.
#Le maximum indique le pique de la courbe d'une exponentiel inversée, (la date supposée du lancement du produit,
# des fois, 1 2 semaines aprés les soldes si le produit est d'une nouvelle collection lancé en periode de soldes).
#Elle ne doit pas être très éloignée de la date de lancement (même semaine, une semaine aprés).
#On utilise ce max parce que des fois, les ventes durant la date de lancement sont faibles, et qu'il nous faut
# le potentiel du produit pour calculer l'appetance

#rc_prix_fab : pour chaque rc, le prix de fabrication, on utilisera pour calculer la marge quand le prix de vente varie. 

df=df[df.week<=9]#par defaut on calcul les ventes sur 9 semaines. On considère ainsi que la "part du lion" des
#ventes à eu lieu dans ce temps. Cela est seulement le cas pour les produits saisonniers

rc_SUM_max={}
rc_prix_fab={}
for rc in unique_rc:
    rc_SUM_max[rc]=df[(df.CODE_REFERENCE==rc) ]["SUM_"].max()#& (df.week==0)
    rc_prix_fab[rc] = (df[(df.CODE_REFERENCE==rc) ]["PRIX_ACTUEL_AVEC_OP"]-df[(df.CODE_REFERENCE==rc) ]["MARGE"]).mean()

In [1334]:
rc_prix_fab

{505536: 39.93,
 507844: 13.849166666666667,
 507938: 38.52,
 507939: 22.62,
 508099: 34.75,
 508530: 30.42,
 508747: 14.345,
 509926: 48.120000000000005,
 511400: 13.589999999999998,
 511852: 35.95,
 514688: 40.69978125,
 516077: 15.467884615384618,
 516088: 29.080000000000002,
 516314: 24.01666666666667,
 516468: 38.195499999999996,
 516472: 28.54,
 516715: 36.8,
 516717: 21.17266081877193,
 516918: 23.310000000000002,
 517069: 19.814999999999998,
 517121: 14.393365384615388,
 517190: 26.893333333333334,
 517197: 23.81,
 517214: 15.051111111111112,
 517250: 22.655595238214286,
 517424: 46.980000000000004,
 517437: 47.95666666666667,
 517523: 18.240384615384613,
 517532: 21.964285714285715,
 517536: 18.01297979787879,
 517540: 14.842440677966103,
 517594: 22.14,
 517638: 34.28583333300001,
 517642: 44.02,
 517745: 19.400000000000002,
 517796: 37.04333333333333,
 518192: 28.899090909090912,
 518208: 40.06,
 518301: 31.410000000000004,
 518350: 28.409999999999997,
 518405: 28.2291666666

# Calcul profit

In [1336]:
#SUM_INITIAL fait reference aux ventes durant la semaine de lancement
df["SUM_INITIAL"]=0
df["SUM_INITIAL"]=df.apply(lambda x:rc_SUM_max[x["CODE_REFERENCE"]] ,axis=1)
df["SUM_INITIAL"]

784        972
795        972
927        972
1401       180
1403       180
         ...  
56146    99144
56151     4536
56152     4536
56153      162
56154      162
Name: SUM_INITIAL, Length: 3738, dtype: int64

In [1337]:
df["PROFIT"] = df["SUM_"]*df["MARGE"]

# Modelisation courbe de profit par temps (Grad desc)

In [1339]:
#ON modélise une courbe (exp inv) pour tous les produit
#Dans une autre version c'est une courbe par cluster de produits

#SUM fait toujour référence au nombre de ventes

import scipy as sc
import math



def expInv(x,alpha_):#Fonction exponentielle inversée utilisée pour modéliser les ventes
    return (x[:,0]*np.exp(-alpha_*x[:,1]))


#example : X=[100,0],[100,1],[100,2],[100,3] avec 100 les ventes initiales durant la semaine de lancement
#et 0,1,2... la semaine
x=df[["SUM_INITIAL","week"]].values
#y : la quantité de ventes pour chaque semaine
y=df["SUM_"].values

# optimize constants for the linear function
alpha_, _ = sc.optimize.curve_fit (expInv, x, y)
alpha_

array([1.94773927])

Nb: La partie plus haut serait normalement executé sur des produits avec historique et plus bas sur des produits sans historique. On racordera ensuite un produit sans historique à un produit avec.
Dans notre cas, on fait le pricing sur le même produits.

# Calcul d'appetances pour un produit

In [1340]:
#l'idée derriere ce calcul est que le prix reflète la valeur estimé d'un produit chez celui qui le commercialise. 
#On utilise le prix pour detecter l'appetance (la valeur moyenne du produit pour une loi normale de consommateurs)

#On regroupe les produits dans 3 clusters. Ceux à prix elevé, à prix moyen et à prix faible.
#l'appetance utilise l'attractivité du produit par rapport à son cluster




from sklearn.cluster import KMeans
km=KMeans(3).fit(df_agg["PRIX_ACTUEL_AVEC_OP"].values.reshape(-1, 1))
df_agg["kmeans"]=km.labels_
appetance={}
for k in range(3):# 
    df_agg_k=df_agg[df_agg.kmeans==k].reset_index(drop=True)
    mediane_cluster=np.median(df_agg_k["SUM_"])
    sums=[]
    sums.append(mediane_cluster)
    
    for i in range(df_agg_k.shape[0]):
        #min et max borne cette appentance entre 0.5 et 1.5 et sqrt lisse l'appetance
        appetance[df_agg_k.loc[i,"CODE_REFERENCE"]] = min(max(math.sqrt(df_agg_k.loc[i,"SUM_"]/mediane_cluster),0.5),1.5)

In [1341]:
appetance

{505536: 0.5,
 507844: 1.5,
 507938: 0.5,
 507939: 0.5,
 508099: 0.5,
 508530: 1.5,
 508747: 1.5,
 509926: 1.0,
 511400: 0.592156525463792,
 511852: 0.5,
 514688: 1.5,
 516077: 1.5,
 516088: 1.5,
 516314: 0.5,
 516468: 1.2649110640673518,
 516472: 0.5,
 516715: 0.5,
 516717: 1.5,
 516918: 1.118033988749895,
 517069: 1.4504813352456845,
 517121: 1.47048891948823,
 517190: 0.5,
 517197: 1.0256451881367417,
 517214: 1.5,
 517250: 1.5,
 517424: 1.0,
 517437: 0.8660254037844386,
 517523: 1.0,
 517532: 0.6620511221285621,
 517536: 1.1078234188139946,
 517540: 1.5,
 517594: 0.5,
 517638: 0.8366600265340756,
 517642: 0.5773502691896257,
 517745: 0.5,
 517796: 0.8660254037844386,
 518192: 0.7416198487095663,
 518208: 1.0,
 518301: 1.0,
 518350: 0.5,
 518405: 1.5,
 518409: 0.8660254037844386,
 518433: 1.5,
 518436: 1.0945555938607907,
 518445: 0.5,
 518459: 0.5,
 518514: 1.5,
 518516: 0.9819805060619657,
 518518: 0.9486832980505138,
 518546: 0.5,
 518745: 1.5,
 518750: 0.7644707871564383,
 51876

In [1342]:
#Dans l'article dont est inspiré ce code, l'appentance est une quantité du même ordre que le prix (evaluée en euro)
#Cette quantité est entrée par le client comme paramètre. 

#N'ayant pas cette possibilité, nous avons calculé une appetance relative au prix initial
#du produit corrigée par (le prix median dans son cluster).
import copy
appentance_unit=copy.deepcopy(appetance)
for k in appetance.keys():
    appetance[k]=appetance[k]*df[df.CODE_REFERENCE == k]["PRIX_ACTUEL_AVEC_OP"].max()
    
    
    
#NB:
#Si on a le prix et les ventes initiales (quelques semaines), on peut estimer cette appetance en 
#1. comparant les qty de ventes avec des produit à prix similaires (au lieu du prix dans la cellule plus haut)
#2. On peut egalement utiliser le stoque initial qui reflète l'appetance estimée des clients sur le produit




In [1343]:
appetance

{505536: 19.995,
 507844: 26.985,
 507938: 19.995,
 507939: 19.995,
 508099: 17.995,
 508530: 59.985,
 508747: 44.985,
 509926: 49.99,
 511400: 10.658817458348256,
 511852: 17.995,
 514688: 68.985,
 516077: 53.985,
 516088: 53.985,
 516314: 17.995,
 516468: 50.5837934520534,
 516472: 22.995,
 516715: 19.995,
 516717: 68.985,
 516918: 44.7101792101083,
 517069: 58.004748596474926,
 517121: 52.9228962123814,
 517190: 19.995,
 517197: 30.75909919222088,
 517214: 53.985,
 517250: 53.985,
 517424: 49.99,
 517437: 43.29260993518409,
 517523: 35.99,
 517532: 30.447731106692572,
 517536: 44.301858518371645,
 517540: 59.985,
 517594: 19.995,
 517638: 38.477994620302134,
 517642: 26.552338880030888,
 517745: 19.995,
 517796: 39.82850832004633,
 518192: 26.690898355057293,
 518208: 49.99,
 518301: 59.99,
 518350: 14.995,
 518405: 68.985,
 518409: 39.82850832004633,
 518433: 74.985,
 518436: 54.71683413710093,
 518445: 24.995,
 518459: 22.995,
 518514: 68.985,
 518516: 39.269400437418014,
 518518:

#  Echantillonage d'une propulation pour optimiser les prix (loi normal)

In [1344]:
#On echantillone un population en utilisant une loi normale.
#Cette population est à 50% "appetante" 50 % "reticente" pour acheter un produit 
#avec une appetance mediane dans le cluster du produit
#la taille de 1000 n'a pas un grand effet sur le prix. Elle defini la granularité (le plus s'est elevé
# le plus la méthode est precise)
#Le scale limite l'etendu de la coube. On estime que la majorité de gens 
#N'ont pas de sentiments "forts" vis-a-vis du produit (on est dans le retail)
population=np.random.normal(loc=1,scale=0.5,size=1000)
population=np.abs(population)

# Optimisation des prix (appetance + courbe de profit)

In [1345]:
#initial_price=prix initial du produit durant la semaine de lancement
# appetance
#population: echantilloné suivant une loie normale
#alpha: paramètre de la courbe exp inv du produit
#NumWeeks : durée de vie du produit

def compute_qty_pred(initial_price,appetance,population,alpha=1.85,numWeeks=9):
    
    r=0
    for p in population:
        #p*appetance*np.exp(-alpha*t) : l'appetance se degradant dans le temps
        
        r+=np.array([0 if p*appetance*np.exp(-alpha*t)-initial_price<0 else 1/(t+1) for t in range(numWeeks)]).sum()
    return r

In [1346]:
#price_range est limité par camaieu
# un array de prix qu'on test, on prend le meilleur (max marge)

optimized_price={}
init_price={}
for k,v in appetance.items():
    if str(k) in set_RC:
        try:
            init_price[k]=df[df.CODE_REFERENCE == k]["PRIX_ACTUEL_AVEC_OP"].max()
            price_range=init_price[k] -0.24*init_price[k] + (np.arange(10)/10)*0.48*init_price[k]
            gain=(price_range -rc_prix_fab[k])*np.array([compute_qty_pred(p,v,population,alpha_) for p in price_range])
            optimized_price[k]=price_range[np.argmax(gain)]
            print("argmax")
            print(np.argmax(gain))
        except:
            None

argmax
9
argmax
4
argmax
9
argmax
7
argmax
9
argmax
7
argmax
9
argmax
9
argmax
9
argmax
9
argmax
1
argmax
7
argmax
9
argmax
6
argmax
9
argmax
1
argmax
1
argmax
9
argmax
9
argmax
4
argmax
9
argmax
9
argmax
9
argmax
3
argmax
9
argmax
9
argmax
9
argmax
9
argmax
2
argmax
9
argmax
9
argmax
6
argmax
9
argmax
9
argmax
0
argmax
8
argmax
8
argmax
1
argmax
9
argmax
9
argmax
1
argmax
9
argmax
9
argmax
7
argmax
9
argmax
7
argmax
9
argmax
9
argmax
0
argmax
0
argmax
5
argmax
7
argmax
3
argmax
9
argmax
9
argmax
9
argmax
8
argmax
9


# Impression des prix optimisés

In [1128]:
file_=codecs.open("results_prix_2","a")

In [1347]:
for k,v in optimized_price.items():
    if str(k) in set_RC:
        print(k)
        print("Initial price")
        print(init_price[k])
        print("optimized")
        print(v)
        print("appetance unit")
        print(appentance_unit[k])
        file_.write(str(k)+","+str(v)+"\n")
        print("----------")

517121
Initial price
35.99
optimized
42.90008
appetance unit
1.47048891948823
----------
517250
Initial price
35.99
optimized
42.90008
appetance unit
1.5
----------
527619
Initial price
23.0
optimized
21.896
appetance unit
0.5
----------
518405
Initial price
45.99
optimized
54.820080000000004
appetance unit
1.5
----------
517638
Initial price
45.99
optimized
50.40504
appetance unit
0.8366600265340756
----------
527113
Initial price
45.99
optimized
37.15992000000001
appetance unit
0.6163360527099886
----------
522660
Initial price
39.99
optimized
47.66808
appetance unit
1.3416407864998738
----------
520205
Initial price
49.99
optimized
59.588080000000005
appetance unit
1.5
----------
518765
Initial price
39.99
optimized
45.748560000000005
appetance unit
0.7071067811865476
----------
527634
Initial price
35.99
optimized
39.445040000000006
appetance unit
1.1338934190276817
----------
519187
Initial price
39.99
optimized
47.66808
appetance unit
1.0
----------
519194
Initial price
45.99
opt